# Naver API 테스트
[홈페이지 상세보기](https://www.ncloud.com/v2/product/applicationService/maps#detail)

![Naver_1](./assets/naver_1.png)
![Naver_2](./assets/naver_2.png)


## 초기화

In [ ]:
import requests
import geocoder
import json
import os
import sys
from dotenv import load_dotenv

def load_credential():
    load_dotenv()
    # 1. 발급받은 키 입력
    cid = os.getenv('NAVER_API_CLIENT_ID')
    secret = os.getenv('NAVER_API_SECRET_KEY')
    assert bool(cid)==True and bool(secret)==True
    return {
        'NAVER_API_CLIENT_ID': cid,
        'NAVER_API_SECRET_KEY': secret
    }

 # 2. 요청 헤더 설정
cred = load_credential()

# 2. 요청 헤더 설정
headers = {
    "X-NCP-APIGW-API-KEY-ID": cred['NAVER_API_CLIENT_ID'],
    "X-NCP-APIGW-API-KEY": cred['NAVER_API_SECRET_KEY'],
}


-------
## Map Direction API Test

In [ ]:

url = "https://maps.apigw.ntruss.com/map-direction/v1/driving"
 
# 4. 내 위치 정보 가져오기
print("📍 현재 내 위치 정보 가져오는 중...")
g = geocoder.ip('me')
 
if g.latlng:
    current_lat, current_lng = g.latlng
    print(f"📍 탐지된 내 위치: 위도 {current_lat}, 경도 {current_lng}")
    print(f"   (주소 추정: {g.address})") # 영문 주소로 나옴
else:
    print("❌ 위치를 찾을 수 없습니다. 기본값(용산역)을 사용합니다.")
    current_lat, current_lng = 37.52989, 126.964775
 
# 내 위치를 출발지로 설정
start_node = f"{current_lng},{current_lat}"
goal_node = "126.9783882,37.5666103"
 
params = {
    "start": start_node, # 내 위치
    "goal": goal_node,  # 서울시청
    "option": "trafast" # 실시간 빠른길
}
 
# 5. 요청 보내기
print("🚗 네이버 지도 API 테스트 중...")
response = requests.get(url, headers=headers, params=params)
 
# 6. 결과 확인
if response.status_code == 200:
    data = response.json()
    if data['code'] == 0:
        summary = data['route']['trafast'][0]['summary']
        distance_km = summary['distance'] / 1000
        duration_min = summary['duration'] / 1000 / 60
       
        print(f"✅ 성공!")
        print(f"출발: 내 위치 ({current_lat}, {current_lng})")
        print(f"도착: 서울시청")
        print(f"📏 거리: {distance_km:.2f} km")
        print(f"⏱️ 시간: 약 {int(duration_min)}분")
    else:
        print("⚠️ 경로 없음:", data['message'])
else:
    print("❌ API 호출 실패:", response.status_code)

-----
## Geocoding API Test


In [ ]:

geocode_endpoint = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
 
# 쿼리 입력
query = "의료 센터"

######################################

params = {
    'query': query,
}

res = requests.get(geocode_endpoint, headers=headers, params=params)
res.raise_for_status()
data = res.json()

print(data)




## Reverse Geocoding Test

In [ ]:


rev_geocode_endpoint = "https://maps.apigw.ntruss.com/map-reversegeocode/v2/gc"
 
g = geocoder.ip('me')
if g.latlng:
    current_lat, current_lng = g.latlng
    print(f"📍 탐지된 내 위치: 위도 {current_lat}, 경도 {current_lng}")
    print(f"   (주소 추정: {g.address})") # 영문 주소로 나옴
else:
    raise Exception("Geocoder로 좌표 찾기 실패")

######################################

# coord_xy = f"{current_lng},{current_lat}"
coord_xy = "126.9783882,37.5666103"
params = {
    "coords": coord_xy,
    "output": "json",
    "orders": "admcode"
}

res = requests.get(rev_geocode_endpoint, headers=headers, params=params)
res.raise_for_status()
print(res.text)
data = res.json()

print(data)
